In [2]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [7]:
featuredDF = pd.read_csv('train3.csv')
featuredDF = pd.get_dummies(featuredDF,columns=['Sex','Embarked','Pclass'])
featuredDF = featuredDF.drop(columns=['Ticket'])
featuredDF['Age'] = featuredDF['Age']/featuredDF['Age'].max()
featuredDF['Fare'] = featuredDF['Fare']/featuredDF['Fare'].max()
featuredDF['SibSp'] = featuredDF['SibSp']/featuredDF['SibSp'].max()
featuredDF['Parch'] = featuredDF['Parch']/featuredDF['Parch'].max()
trainDF = featuredDF[featuredDF['Cabin'].notna()]
testDF = featuredDF[featuredDF['Cabin'].isna()]
trainDF

,PassengerId,Survived,Age,SibSp,Parch,Fare,Name,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
1,2,1.0,0.4750,0.125,0.000000,0.139136,0.782000,C,True,False,True,False,False,True,False,False
3,4,1.0,0.4375,0.125,0.000000,0.103644,0.782000,C,True,False,False,False,True,True,False,False
6,7,0.0,0.6750,0.000,0.000000,0.101229,0.156673,E,False,True,False,False,True,True,False,False
10,11,1.0,0.0500,0.125,0.111111,0.032596,0.697802,G,True,False,False,False,True,False,False,True
11,12,1.0,0.7250,0.000,0.000000,0.051822,0.697802,C,True,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,1296,NaN,0.5375,0.125,0.000000,0.054107,0.156673,D,False,True,True,False,False,True,False,False
1296,1297,NaN,0.2500,0.000,0.000000,0.027058,0.156673,D,False,True,True,False,False,False,True,False
1298,1299,NaN,0.6250,0.125,0.111111,0.412821,0.156673,C,False,True,True,False,False,True,False,False
1302,1303,NaN,0.4625,0.125,0.000000,0.175668,0.782000,C,True,False,False,True,False,True,False,False


In [13]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, DF:pd.DataFrame):
        self.cabin= pd.get_dummies(DF['Cabin']).values
        DF = DF.drop(columns=['Cabin','PassengerId','Survived'])
        self.data = DF.astype(float).values
        pass

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        x = torch.FloatTensor(self.data[idx])
        y = torch.FloatTensor(self.cabin[idx])
        return x, y


dataSet = MyDataset(DF=trainDF)
testSet = MyDataset(DF=testDF)
dataSet[0]

(tensor([0.4750, 0.1250, 0.0000, 0.1391, 0.7820, 1.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000]),
 tensor([0., 0., 1., 0., 0., 0., 0., 0.]))

In [14]:

trainSet, valSet = torch.utils.data.random_split(dataSet,(0.8,0.2))

In [15]:
trainLoader= torch.utils.data.DataLoader(trainSet,batch_size=2048,sampler=torch.utils.data.RandomSampler(trainSet))
valLoader= torch.utils.data.DataLoader(valSet,batch_size=2048,sampler=torch.utils.data.RandomSampler(valSet))
testLoader = torch.utils.data.DataLoader(testSet,batch_size=2048)
next(iter(trainLoader))

[tensor([[0.9500, 0.1250, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
         [0.1500, 0.2500, 0.1111,  ..., 0.0000, 1.0000, 0.0000],
         [0.3875, 0.1250, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
         ...,
         [0.4125, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
         [0.5250, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
         [0.6250, 0.1250, 0.0000,  ..., 1.0000, 0.0000, 0.0000]]),
 tensor([[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])]

In [16]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(13, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 7),
            torch.nn.ReLU(),
            torch.nn.Linear(7, 4),
            torch.nn.ReLU(),
            torch.nn.Linear(4, 2)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=7, bias=True)
    (3): ReLU()
    (4): Linear(in_features=7, out_features=4, bias=True)
    (5): ReLU()
    (6): Linear(in_features=4, out_features=2, bias=True)
  )
)


In [17]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters())

In [18]:
def trainAI(dataloader, model, loss_fn, optimizer):
    model.train()
    trainLoss=0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        trainLoss +=loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return trainLoss

def valAI(dataloader, model, loss_fn):
    model.eval()
    valLoss =0
    with torch.no_grad():
        for X ,y  in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            # for i in loss:
            valLoss+= loss.item()
    return valLoss

trainLoss=[]
valLoss=[]
bestModel = model
bestLoss = float('inf')
cnt=0

while(cnt<100):
    trainLoss.append(trainAI(trainLoader, model, loss_fn, optimizer))
    valLoss.append(valAI(valLoader, model, loss_fn))

    print(f'cnt: {cnt} - valLoss: {valLoss[-1]} - trainLoss: {trainLoss[-1]}')
    if bestLoss<valLoss[-1]:
        cnt+=1
    else:
        cnt = 0
        bestLoss = valLoss[-1]
        bestModel = model

plt.plot(trainLoss,label='trainLoss')
plt.plot(valLoss,label='valLoss')
plt.legend()
plt.close()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [23]:
def testAI(dataloader, model):
    model.eval()
    result = []
    y=892
    with torch.no_grad():
        for X ,_  in dataloader:
            X  = X.to(device)
            pred = model(X)
            for i in pred:
                result.append([y,torch.argmax(i).item()])
                y+=1
    return result

result = testAI(testLoader, bestModel)
result = pd.DataFrame(result)
result = result.astype(int)
result.columns=['passengerId','Survived']
result

,passengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [24]:
result.to_csv('result.csv',index=False)